In [7]:
!pip install re


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [8]:
import openai
import re
import httpx

In [19]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""" .strip()

In [ ]:
class ChatBot:
    def __init__(self, system=""):
        self.client = openai.OpenAI(
            api_key="EMPTY",
            base_url=""
        )
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = self.client.chat.completions.create(model="Qwen/Qwen3-14B-AWQ", messages=self.messages)
        print(completion.usage)
        return completion.choices[0].message.content

In [21]:
action_re = re.compile(r"^Action: (\w+): (.*)$")

def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]

def calculate(what):
    return eval(what)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
}

def query(question, max_turns=5):
    i = 0
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print("-- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation: {}".format(observation))
            next_prompt = "Observation: {}".format(observation)
        else:
            return


In [26]:
query("Where is lebanon located and what are the tourist attractions? and what is 10 + 6")

AttributeError: 'str' object has no attribute 'usage'

In [24]:
calculate("3+7")

10

In [23]:
wikipedia("Large language models")

'months. Foundation <span class="searchmatch">models</span> List of <span class="searchmatch">large</span> <span class="searchmatch">language</span> <span class="searchmatch">models</span> List of chatbots <span class="searchmatch">Language</span> <span class="searchmatch">model</span> benchmark Reinforcement learning Small <span class="searchmatch">language</span> <span class="searchmatch">model</span> Brown, Tom B.;'

In [25]:
query("What is bmw tech office munich?")

AttributeError: 'str' object has no attribute 'usage'